In [47]:
import shutil
from deepCR import deepCR
from astropy.io import fits
import matplotlib.pyplot as plt
from matplotlib import cm
import h5py
import numpy as np
import os
import gc
import glob as glob
import shutil
from astropy.io import ascii
from astropy.table import Table
from astropy.visualization import ZScaleInterval
from astroquery.mast import Observations
from ccdproc import ImageFileCollection
from drizzlepac import tweakreg
from drizzlepac import astrodrizzle
from IPython.display import Image
%matplotlib inline

## F435W

obs_table = Observations.query_criteria(obs_collection='HST',
                                            filters='F435W',
                                            instrument_name='ACS/WFC')

calib_mask = obs_table['calib_level'] == 3
obs_table = obs_table[calib_mask]
proposal_id = obs_table['proposal_id']
prop_num = len(proposal_id)
np.random.seed(56)
id_idx = np.random.randint(0,prop_num-1,size = 15)
chosen_prop_ids = proposal_id[id_idx]
f435w_prop_ids = chosen_prop_ids

In [26]:
f435_prop_ids = [10595,9442,10760,10631,12103,13498,13504]

* 10595 (j9es02010, j9es07010)
* 9442 (j6lp01010, j6lp03010, j6lp05010)
* 10760 (j9ju02010, j9ju04010)
* 10631 (j9df03010, j9df01010, j9df04010)
* 12103 (jbfla3010)
* 13498 (jc8mb1010)
* 13504 (jcdub1010, jcdub4010, jcdu98020)

## F606W

obs_table = Observations.query_criteria(obs_collection='HST',
                                            filters='F606W',
                                            instrument_name='ACS/WFC')

calib_mask = obs_table['calib_level'] == 3
obs_table = obs_table[calib_mask]
proposal_id = obs_table['proposal_id']
prop_num = len(proposal_id)
np.random.seed(163)
id_idx = np.random.randint(0,prop_num-1,size = 25)
chosen_prop_ids = proposal_id[id_idx]
f606w_prop_ids = chosen_prop_ids

In [27]:
f606_prop_ids = [9803,14343,13057,13018,13375,10128,13860,15275]

* 9803 (j8mt02010, j8mt03010, j8mt09010)
* 14343 (jcxx01010, jcxx06010)
* 13057 (jc3f01010)
* 9018 (j8c061vnq)
* 13018 (jc3a01010, jc3a02010, jc3a05010)
* 13375 (jc9704010, jc9707010)
* 10128 (j91m01010, j91m03010)
* 13860 (jckh03010, jckh04010)
* 15275 (jdn002020)

## F814W

obs_table = Observations.query_criteria(obs_collection='HST',
                                            filters='F814W',
                                            instrument_name='ACS/WFC')

calib_mask = obs_table['calib_level'] == 3
obs_table = obs_table[calib_mask]
proposal_id = obs_table['proposal_id']
prop_num = len(proposal_id)
np.random.seed(216)
id_idx = np.random.randint(0,prop_num-1,size = 25)
chosen_prop_ids = proposal_id[id_idx]
f814w_prop_ids = chosen_prop_ids

* 13024 (jbz010010, jbz016010)
* 10235 (j8xz15020, j8xz16020)
* 10092 (j8xi01010)
* 13691 (jcnw11010, jcnw13010)
* 15336 (jdjf57010, jdjf10020)
* 12058 (jbf306010, jbf310010, jbf312010)
* 14610 (jdb612010, jdb617010)
* 13737 (jcoy02010)

In [28]:
f814_prop_ids = [13024,10235,10092,13691,15336,12058,14610,13737]

In [2]:
f435w_obs_ids = ['j9es02010', 'j9es07010', 'j6lp01010',
                 'j6lp03010', 'j6lp05010', 'j9ju02010',
                 'j9ju04010', 'j9df03010', 'j9df01010',
                 'j9df04010', 'jbfla3010', 'jc8mb1010',
                 'jcdub1010', 'jcdub4010', 'jcdu98020']

f606w_obs_ids = ['j8mt02010', 'j8mt03010', 'j8mt09010',
                 'jcxx01010', 'jcxx06010', 'jc3f01010',
                 'jc3a01010', 'jc3a02010', 'jc3a05010', 
                 'jc9704010', 'jc9707010', 'j91m01010',
                 'j91m03010', 'jckh03010', 'jckh04010',
                 'jdn002020']

f814w_obs_ids = ['jbz010010', 'jbz016010', 'j8xz15020',
                 'j8xz16020', 'j8xi01010', 'jcnw11010',
                 'jcnw13010', 'jdjf57010', 'jdjf10020',
                 'jbf306010', 'jbf310010', 'jbf312010',
                 'jdb612010', 'jdb617010', 'jcoy02010']

In [3]:
f435w_Table = Observations.query_criteria(obstype='all',
                                      obs_collection='HST',
                                      obs_id = f435w_obs_ids)
f606w_Table = Observations.query_criteria(obstype='all',
                                      obs_collection='HST',
                                      obs_id = f606w_obs_ids)
f814w_Table = Observations.query_criteria(obstype='all',
                                      obs_collection='HST',
                                      obs_id = f814w_obs_ids)

In [4]:
f435w_product = Observations.get_product_list(f435w_Table)
Observations.download_products(f435w_product,
                              download_dir = '/Users/jameskwon/Documents/Research/Data/deepcr/f435w',
                              mrp_only = False,
                              productSubGroupDescription='FLC')
f606w_product = Observations.get_product_list(f606w_Table)
Observations.download_products(f606w_product,
                              download_dir = '/Users/jameskwon/Documents/Research/Data/deepcr/f606w',
                              mrp_only = False,
                              productSubGroupDescription='FLC')
f814w_product = Observations.get_product_list(f814w_Table)
Observations.download_products(f814w_product,
                              download_dir = '/Users/jameskwon/Documents/Research/Data/deepcr/f814w',
                              mrp_only = False,
                              productSubGroupDescription='FLC')

Local Path,Status,Message,URL
str99,str8,object,object
/Users/jameskwon/Documents/Research/Data/deepcr/f814w/mastDownload/HST/j8xi01vxq/j8xi01vxq_flc.fits,COMPLETE,None,None
/Users/jameskwon/Documents/Research/Data/deepcr/f814w/mastDownload/HST/j8xi01w0q/j8xi01w0q_flc.fits,COMPLETE,None,None
/Users/jameskwon/Documents/Research/Data/deepcr/f814w/mastDownload/HST/j8xi01w3q/j8xi01w3q_flc.fits,COMPLETE,None,None
/Users/jameskwon/Documents/Research/Data/deepcr/f814w/mastDownload/HST/j8xi01w7q/j8xi01w7q_flc.fits,COMPLETE,None,None
/Users/jameskwon/Documents/Research/Data/deepcr/f814w/mastDownload/HST/j8xz15bhq/j8xz15bhq_flc.fits,COMPLETE,None,None
/Users/jameskwon/Documents/Research/Data/deepcr/f814w/mastDownload/HST/j8xz15bjq/j8xz15bjq_flc.fits,COMPLETE,None,None
/Users/jameskwon/Documents/Research/Data/deepcr/f814w/mastDownload/HST/j8xz16bsq/j8xz16bsq_flc.fits,COMPLETE,None,None
/Users/jameskwon/Documents/Research/Data/deepcr/f814w/mastDownload/HST/j8xz16buq/j8xz16buq_flc.fits,COMPLETE,None,None
/Users/jameskwon/Documents/Research/Data/deepcr/f814w/mastDownload/HST/jbf306noq/jbf306noq_flc.fits,COMPLETE,None,None


In [21]:
base_dir = os.environ.get('DEEPCR_DIR')
f606_dir = os.path.join(base_dir,'f606w')
f435_dir = os.path.join(base_dir,'f435w')
f814_dir = os.path.join(base_dir,'f814w')

f606_mast_dir = os.path.join(f606_dir,'mastDownload','HST')
f435_mast_dir = os.path.join(f435_dir,'mastDownload','HST')
f814_mast_dir = os.path.join(f814_dir,'mastDownload','HST')

In [37]:
for prop_id in f606_prop_ids:
    dir_exist = os.path.isdir(base_dir+f'/f606w/{prop_id}')
    if not dir_exist:
        os.mkdir(base_dir+f'/f606w/{prop_id}')

In [41]:
f606_obs_table = Observations.query_criteria(obs_collection='HST', 
                                        filters='F606W', 
                                        instrument_name='ACS/WFC')

In [38]:
for prop_id in f435_prop_ids:
    dir_exist = os.path.isdir(base_dir+f'/f435w/{prop_id}')
    if not dir_exist:
        os.mkdir(base_dir+f'/f435w/{prop_id}')

In [44]:
f435_obs_table = Observations.query_criteria(obs_collection='HST', 
                                        filters='F435W', 
                                        instrument_name='ACS/WFC')

In [39]:
for prop_id in f814_prop_ids:
    dir_exist = os.path.isdir(base_dir+f'/f814w/{prop_id}')
    if not dir_exist:
        os.mkdir(base_dir+f'/f814w/{prop_id}')

In [45]:
f814_obs_table = Observations.query_criteria(obs_collection='HST', 
                                        filters='F814W', 
                                        instrument_name='ACS/WFC')

In [117]:
for f_download in os.listdir(f606_mast_dir):
    sub_dir = os.path.join(f606_mast_dir,f_download)
    if os.path.isdir(sub_dir):
        file = os.listdir(sub_dir)
        if len(file) != 1:
            print(sub_dir)
            print(file)
            raise ValueError
        else:

            file = file[0]
            print(file)
            file_dir = os.path.join(sub_dir,file)
            if os.path.isfile(file_dir):
                idx = (file.index('_flc.fits'))
                for i,ID in enumerate(f606_obs_table['obs_id']):
                    if file[:idx-5] in ID:
                        break
                prop_id = f606_obs_table['proposal_id'][i]
                src = file_dir
                print(os.path.join(f606_dir,prop_id))
                if os.path.isdir(os.path.join(f606_dir,prop_id)):
                    dst = os.path.join(f606_dir,prop_id,file)
                    shutil.move(src,dst)
                else:
                    raise ValueError

j8mt09siq_flc.fits
/Users/jameskwon/Documents/Research/Data/deepcr/f606w/9803
jc3f01b8q_flc.fits
/Users/jameskwon/Documents/Research/Data/deepcr/f606w/13057
jc9707rzq_flc.fits
/Users/jameskwon/Documents/Research/Data/deepcr/f606w/13375
j8mt02deq_flc.fits
/Users/jameskwon/Documents/Research/Data/deepcr/f606w/9803
j91m03p9q_flc.fits
/Users/jameskwon/Documents/Research/Data/deepcr/f606w/10128
jcxx01bbq_flc.fits
/Users/jameskwon/Documents/Research/Data/deepcr/f606w/14343
jdn002bxq_flc.fits
/Users/jameskwon/Documents/Research/Data/deepcr/f606w/15275
jc3a01ioq_flc.fits
/Users/jameskwon/Documents/Research/Data/deepcr/f606w/13018
jcxx01b0q_flc.fits
/Users/jameskwon/Documents/Research/Data/deepcr/f606w/14343
jc9704dxq_flc.fits
/Users/jameskwon/Documents/Research/Data/deepcr/f606w/13375
jc9704drq_flc.fits
/Users/jameskwon/Documents/Research/Data/deepcr/f606w/13375
jckh04rqq_flc.fits
/Users/jameskwon/Documents/Research/Data/deepcr/f606w/13860
jckh03r0q_flc.fits
/Users/jameskwon/Documents/Research

In [119]:
for f_download in os.listdir(f814_mast_dir):
    sub_dir = os.path.join(f814_mast_dir,f_download)
    if os.path.isdir(sub_dir):
        file = os.listdir(sub_dir)
        if len(file) != 1:
            print(sub_dir)
            print(file)
            raise ValueError
        else:

            file = file[0]
            print(file)
            file_dir = os.path.join(sub_dir,file)
            if os.path.isfile(file_dir):
                idx = (file.index('_flc.fits'))
                for i,ID in enumerate(f814_obs_table['obs_id']):
                    if file[:idx-5] in ID:
                        break
                prop_id = f814_obs_table['proposal_id'][i]
                src = file_dir
                print(os.path.join(f814_dir,prop_id))
                if os.path.isdir(os.path.join(f814_dir,prop_id)):
                    dst = os.path.join(f814_dir,prop_id,file)
                    shutil.move(src,dst)
                else:
                    raise ValueError

jdb617okq_flc.fits
/Users/jameskwon/Documents/Research/Data/deepcr/f814w/14610
jbf306nyq_flc.fits
/Users/jameskwon/Documents/Research/Data/deepcr/f814w/12058
jbz010doq_flc.fits
/Users/jameskwon/Documents/Research/Data/deepcr/f814w/13024
jcnw13o2q_flc.fits
/Users/jameskwon/Documents/Research/Data/deepcr/f814w/13691
j8xi01vxq_flc.fits
/Users/jameskwon/Documents/Research/Data/deepcr/f814w/10092
jdjf10e4q_flc.fits
/Users/jameskwon/Documents/Research/Data/deepcr/f814w/15336
jbf312f6q_flc.fits
/Users/jameskwon/Documents/Research/Data/deepcr/f814w/12058
j8xi01w0q_flc.fits
/Users/jameskwon/Documents/Research/Data/deepcr/f814w/10092
jcoy02c8q_flc.fits
/Users/jameskwon/Documents/Research/Data/deepcr/f814w/13737
jcnw13nzq_flc.fits
/Users/jameskwon/Documents/Research/Data/deepcr/f814w/13691
j8xz15bhq_flc.fits
/Users/jameskwon/Documents/Research/Data/deepcr/f814w/10235
j8xi01w3q_flc.fits
/Users/jameskwon/Documents/Research/Data/deepcr/f814w/10092
jcnw11muq_flc.fits
/Users/jameskwon/Documents/Resear

In [121]:
for f_download in os.listdir(f435_mast_dir):
    sub_dir = os.path.join(f435_mast_dir,f_download)
    if os.path.isdir(sub_dir):
        file = os.listdir(sub_dir)
        if len(file) != 1:
            print(sub_dir)
            print(file)
            raise ValueError
        else:

            file = file[0]
            print(file)
            file_dir = os.path.join(sub_dir,file)
            if os.path.isfile(file_dir):
                idx = (file.index('_flc.fits'))
                for i,ID in enumerate(f435_obs_table['obs_id']):
                    if file[:idx-5] in ID:
                        break
                prop_id = f435_obs_table['proposal_id'][i]
                src = file_dir
                print(os.path.join(f435_dir,prop_id))
                if os.path.isdir(os.path.join(f435_dir,prop_id)):
                    dst = os.path.join(f435_dir,prop_id,file)
                    shutil.move(src,dst)
                else:
                    raise ValueError

j9ju04evq_flc.fits
/Users/jameskwon/Documents/Research/Data/deepcr/f435w/10760
j9ju02hyq_flc.fits
/Users/jameskwon/Documents/Research/Data/deepcr/f435w/10760
j9es07smq_flc.fits
/Users/jameskwon/Documents/Research/Data/deepcr/f435w/10595
j9df04idq_flc.fits
/Users/jameskwon/Documents/Research/Data/deepcr/f435w/10631
jcdub4blq_flc.fits
/Users/jameskwon/Documents/Research/Data/deepcr/f435w/13504
j9ju02i8q_flc.fits
/Users/jameskwon/Documents/Research/Data/deepcr/f435w/10760
j9ju04f1q_flc.fits
/Users/jameskwon/Documents/Research/Data/deepcr/f435w/10760
j9df01jqq_flc.fits
/Users/jameskwon/Documents/Research/Data/deepcr/f435w/10631
j6lp01egq_flc.fits
/Users/jameskwon/Documents/Research/Data/deepcr/f435w/9442
j9ju02i2q_flc.fits
/Users/jameskwon/Documents/Research/Data/deepcr/f435w/10760
jcdub4bfq_flc.fits
/Users/jameskwon/Documents/Research/Data/deepcr/f435w/13504
j9es02kvq_flc.fits
/Users/jameskwon/Documents/Research/Data/deepcr/f435w/10595
j6lp01edq_flc.fits
/Users/jameskwon/Documents/Researc

In [144]:
from drizzlepac import astrodrizzle

In [149]:
repo_dir = os.environ.get('DEEPCR')
repo_dir = os.path.join(repo_dir,'codes')

In [196]:
def remove_extra(data_dir):
    repo_dir = os.environ.get('DEEPCR')
    repo_dir = os.path.join(repo_dir,'codes')
    rm_list = []
    for file in os.listdir(repo_dir):
        if 'OrIg_' in file:
            pass
        elif 'final_' in file:
            rm_list.append(file)
        elif '_sci2.fits' in file:
            rm_list.append(file)
        elif '_sci1.fits' in file:
            rm_list.append(file)
        elif '_staticMask.fits' in file:
            rm_list.append(file)
        elif 'astrodrizzle.log' in file:
            rm_list.append(file)
    a = os.listdir(repo_dir)
    
    for f_rm in rm_list:
        rm_dir = os.path.join(repo_dir,f_rm)
        os.remove(rm_dir)
    
    rm_list = []
    for file in os.listdir(data_dir):
        if 'blt.fits' in file:
            rm_list.append(file)
        elif '_final_mask.fits' in file:
            rm_list.append(file)
        
        elif '_single_mask.fits' in file:
            rm_list.append(file)
        
        elif '_single_sci.fits' in file:
            rm_list.append(file)
        
        elif '_single_wht.fits' in file:
            rm_list.append(file)

    for f_rm in rm_list:
        rm_dir = os.path.join(data_dir,f_rm)
        os.remove(rm_dir)

In [197]:
for prop_id in os.listdir(f606_dir):
    sub_dir = os.path.join(f606_dir,prop_id)
    if os.path.isdir(sub_dir):
        if prop_id != 'mastDownload':
            for visit_num in os.listdir(sub_dir):
                if visit_num != '.DS_Store':
                    subsub_dir = os.path.join(sub_dir,visit_num)
                    drizzle_path = subsub_dir + '/*flc.fits'
                    astrodrizzle.AstroDrizzle(drizzle_path)
                    remove_extra(subsub_dir)

Setting up logfile :  astrodrizzle.log
AstroDrizzle Version 3.1.6 (2020-02-19 16:16:30 -0500) started at: 17:54:54.665 (28/06/2020)

==== Processing Step  Initialization  started at  17:54:54.668 (28/06/2020)

WCS Keywords

Number of WCS axes: 2
CTYPE : 'RA---TAN'  'DEC--TAN'  
CRVAL : 53.17421639910745  -27.912380482584  
CRPIX : 2107.0  2117.5  
CD1_1 CD1_2  : 9.674955568087742e-07  -1.3855150194619202e-05  
CD2_1 CD2_2  : -1.3855150194619202e-05  -9.674955568087742e-07  
NAXIS : 4214  4235
********************************************************************************
*
*  Estimated memory usage:  up to 1072 Mb.
*  Output image size:       4214 X 4235 pixels. 
*  Output image file:       ~ 204 Mb. 
*  Cores available:         4
*
********************************************************************************
==== Processing Step Initialization finished at 17:54:55.493 (28/06/2020)
==== Processing Step  Static Mask  started at  17:54:55.496 (28/06/2020)

==== Processing Step Static


AstroDrizzle Version 3.1.6 is finished processing at 17:55:28.255 (28/06/2020).



   --------------------          --------------------
                   Step          Elapsed time
   --------------------          --------------------

         Initialization          0.8255 sec.
            Static Mask          0.9484 sec.
           Subtract Sky          2.3137 sec.
       Separate Drizzle          3.6041 sec.
          Create Median          5.0262 sec.
                   Blot          5.9516 sec.
                Driz_CR          3.2212 sec.
          Final Drizzle          11.6748 sec.
   ====================          ====================
                  Total          33.5654 sec.

Trailer file written to:  astrodrizzle.log
Setting up logfile :  astrodrizzle.log
AstroDrizzle Version 3.1.6 (2020-02-19 16:16:30 -0500) started at: 17:55:28.380 (28/06/2020)

==== Processing Step  Initialization  started at  17:55:28.383 (28/06/2020)

WCS Keywords

Number of WCS axes: 2
CTYPE : 'R

==== Processing Step Driz_CR finished at 17:55:49.065 (28/06/2020)
==== Processing Step  Final Drizzle  started at  17:55:49.074 (28/06/2020)

WCS Keywords

Number of WCS axes: 2
CTYPE : 'RA---TAN'  'DEC--TAN'  
CRVAL : 53.1741662974592  -27.911071341884355  
CRPIX : 2106.0  2119.0  
CD1_1 CD1_2  : 9.674949350685262e-07  -1.385515023803488e-05  
CD2_1 CD2_2  : -1.385515023803488e-05  -9.674949350685262e-07  
NAXIS : 4212  4238
-Generating simple FITS output: final_drc_sci.fits
Writing out image to disk: final_drc_sci.fits
Writing out image to disk: final_drc_wht.fits
Writing out image to disk: final_drc_ctx.fits
==== Processing Step Final Drizzle finished at 17:55:59.582 (28/06/2020)

AstroDrizzle Version 3.1.6 is finished processing at 17:55:59.58 (28/06/2020).



   --------------------          --------------------
                   Step          Elapsed time
   --------------------          --------------------

         Initialization          0.9029 sec.
            Static Mask 

Writing out image to disk: /Users/jameskwon/Documents/Research/Data/deepcr/f606w/9803/2/j8mt02deq_sci2_blt.fits
==== Processing Step Blot finished at 17:56:16.383 (28/06/2020)
==== Processing Step  Driz_CR  started at  17:56:16.384 (28/06/2020)

Creating output: /Users/jameskwon/Documents/Research/Data/deepcr/f606w/9803/2/j8mt02dbq_sci1_crmask.fits
Creating output: /Users/jameskwon/Documents/Research/Data/deepcr/f606w/9803/2/j8mt02deq_sci1_crmask.fits
Creating output: /Users/jameskwon/Documents/Research/Data/deepcr/f606w/9803/2/j8mt02dbq_sci2_crmask.fits
Creating output: /Users/jameskwon/Documents/Research/Data/deepcr/f606w/9803/2/j8mt02deq_sci2_crmask.fits
==== Processing Step Driz_CR finished at 17:56:19.401 (28/06/2020)
==== Processing Step  Final Drizzle  started at  17:56:19.410 (28/06/2020)

WCS Keywords

Number of WCS axes: 2
CTYPE : 'RA---TAN'  'DEC--TAN'  
CRVAL : 53.16511271933705  -27.920486356655545  
CRPIX : 2105.5  2120.5  
CD1_1 CD1_2  : 9.664546559966784e-07  -1.3855222

    Blot: creating blotted image:  /Users/jameskwon/Documents/Research/Data/deepcr/f606w/13018/1/jc3a01ioq_flc.fits[sci,1]
Using default C-based coordinate transformation...
-Generating simple FITS output: /Users/jameskwon/Documents/Research/Data/deepcr/f606w/13018/1/jc3a01ioq_sci1_blt.fits
Writing out image to disk: /Users/jameskwon/Documents/Research/Data/deepcr/f606w/13018/1/jc3a01ioq_sci1_blt.fits
    Blot: creating blotted image:  /Users/jameskwon/Documents/Research/Data/deepcr/f606w/13018/1/jc3a01ioq_flc.fits[sci,2]
Using default C-based coordinate transformation...
-Generating simple FITS output: /Users/jameskwon/Documents/Research/Data/deepcr/f606w/13018/1/jc3a01ioq_sci2_blt.fits
Writing out image to disk: /Users/jameskwon/Documents/Research/Data/deepcr/f606w/13018/1/jc3a01ioq_sci2_blt.fits
==== Processing Step Blot finished at 17:56:46.77 (28/06/2020)
==== Processing Step  Driz_CR  started at  17:56:46.777 (28/06/2020)

Creating output: /Users/jameskwon/Documents/Research/Data

...


-----  User specified keywords:  -----
       Sky Value Keyword:  'MDRIZSKY'
       Data Units Keyword: 'BUNIT'


-----  Input file list:  -----

   **  Input image: 'jcxx01b0q_flc.fits'
       EXT: 'SCI',1;	MASK: jcxx01b0q_skymatch_mask_sci1.fits[0]
       EXT: 'SCI',2;	MASK: jcxx01b0q_skymatch_mask_sci2.fits[0]

   **  Input image: 'jcxx01b2q_flc.fits'
       EXT: 'SCI',1;	MASK: jcxx01b2q_skymatch_mask_sci1.fits[0]
       EXT: 'SCI',2;	MASK: jcxx01b2q_skymatch_mask_sci2.fits[0]

   **  Input image: 'jcxx01b5q_flc.fits'
       EXT: 'SCI',1;	MASK: jcxx01b5q_skymatch_mask_sci1.fits[0]
       EXT: 'SCI',2;	MASK: jcxx01b5q_skymatch_mask_sci2.fits[0]

   **  Input image: 'jcxx01bbq_flc.fits'
       EXT: 'SCI',1;	MASK: jcxx01bbq_skymatch_mask_sci1.fits[0]
       EXT: 'SCI',2;	MASK: jcxx01bbq_skymatch_mask_sci2.fits[0]

   **  Input image: 'jcxx01bfq_flc.fits'
       EXT: 'SCI',1;	MASK: jcxx01bfq_skymatch_mask_sci1.fits[0]
       EXT: 'SCI',2;	MASK: jcxx01bfq_skymatch_mask_sci2.fits[0]

   

reference sky value for image '/Users/jameskwon/Documents/Research/Data/deepcr/f606w/14343/1/jcxx01b2q_flc.fits' is 25.72148895263672
reference sky value for image '/Users/jameskwon/Documents/Research/Data/deepcr/f606w/14343/1/jcxx01b5q_flc.fits' is 25.289215087890625
reference sky value for image '/Users/jameskwon/Documents/Research/Data/deepcr/f606w/14343/1/jcxx01bbq_flc.fits' is 45.970191955566406
reference sky value for image '/Users/jameskwon/Documents/Research/Data/deepcr/f606w/14343/1/jcxx01bfq_flc.fits' is 28.540328979492188
reference sky value for image '/Users/jameskwon/Documents/Research/Data/deepcr/f606w/14343/1/jcxx01biq_flc.fits' is 26.386600494384766
Saving output median image to: 'final_med.fits'
==== Processing Step Create Median finished at 18:02:43.390 (28/06/2020)
==== Processing Step  Blot  started at  18:02:43.391 (28/06/2020)

    Blot: creating blotted image:  /Users/jameskwon/Documents/Research/Data/deepcr/f606w/14343/1/jcxx01b0q_flc.fits[sci,1]
Using default C

       Separate Drizzle          5.3414 sec.
          Create Median          12.3003 sec.
                   Blot          16.9383 sec.
                Driz_CR          6.4529 sec.
          Final Drizzle          29.7504 sec.
   ====================          ====================
                  Total          81.4265 sec.

Trailer file written to:  astrodrizzle.log
Setting up logfile :  astrodrizzle.log
AstroDrizzle Version 3.1.6 (2020-02-19 16:16:30 -0500) started at: 18:03:36.912 (28/06/2020)

==== Processing Step  Initialization  started at  18:03:36.914 (28/06/2020)

WCS Keywords

Number of WCS axes: 2
CTYPE : 'RA---TAN'  'DEC--TAN'  
CRVAL : 75.51215209350015  -66.39156579657443  
CRPIX : 2109.5  2124.0  
CD1_1 CD1_2  : -8.906183865926035e-06  1.0657444511430497e-05  
CD2_1 CD2_2  : 1.0657444511430497e-05  8.906183865926035e-06  
NAXIS : 4219  4248
********************************************************************************
*
*  Estimated memory usage:  up to 1076 Mb.
*  O

CD1_1 CD1_2  : -8.906183865926035e-06  1.0657444511430497e-05  
CD2_1 CD2_2  : 1.0657444511430497e-05  8.906183865926035e-06  
NAXIS : 4219  4248
-Generating simple FITS output: final_drc_sci.fits
Writing out image to disk: final_drc_sci.fits
Writing out image to disk: final_drc_wht.fits
Writing out image to disk: final_drc_ctx.fits
==== Processing Step Final Drizzle finished at 18:04:06.898 (28/06/2020)

AstroDrizzle Version 3.1.6 is finished processing at 18:04:06.900 (28/06/2020).



   --------------------          --------------------
                   Step          Elapsed time
   --------------------          --------------------

         Initialization          0.8177 sec.
            Static Mask          0.8777 sec.
           Subtract Sky          1.9544 sec.
       Separate Drizzle          2.9312 sec.
          Create Median          4.2837 sec.
                   Blot          5.5376 sec.
                Driz_CR          2.9805 sec.
          Final Drizzle          10.58

==== Processing Step Separate Drizzle finished at 18:04:17.892 (28/06/2020)
==== Processing Step  Create Median  started at  18:04:17.896 (28/06/2020)

reference sky value for image '/Users/jameskwon/Documents/Research/Data/deepcr/f606w/13860/3/jckh03qyq_flc.fits' is 96.44960021972656
reference sky value for image '/Users/jameskwon/Documents/Research/Data/deepcr/f606w/13860/3/jckh03r0q_flc.fits' is 158.72683715820312
reference sky value for image '/Users/jameskwon/Documents/Research/Data/deepcr/f606w/13860/3/jckh03r4q_flc.fits' is 128.28396606445312
reference sky value for image '/Users/jameskwon/Documents/Research/Data/deepcr/f606w/13860/3/jckh03r7q_flc.fits' is 147.3824920654297
Saving output median image to: 'final_med.fits'
==== Processing Step Create Median finished at 18:04:25.788 (28/06/2020)
==== Processing Step  Blot  started at  18:04:25.789 (28/06/2020)

    Blot: creating blotted image:  /Users/jameskwon/Documents/Research/Data/deepcr/f606w/13860/3/jckh03qyq_flc.fits[sci,1]

In [198]:
for prop_id in os.listdir(f435_dir):
    sub_dir = os.path.join(f435_dir,prop_id)
    if os.path.isdir(sub_dir):
        if prop_id != 'mastDownload':
            for visit_num in os.listdir(sub_dir):
                if visit_num != '.DS_Store':
                    subsub_dir = os.path.join(sub_dir,visit_num)
                    drizzle_path = subsub_dir + '/*flc.fits'
                    astrodrizzle.AstroDrizzle(drizzle_path)
                    remove_extra(subsub_dir)

Setting up logfile :  astrodrizzle.log
AstroDrizzle Version 3.1.6 (2020-02-19 16:16:30 -0500) started at: 18:06:15.4 (28/06/2020)

==== Processing Step  Initialization  started at  18:06:15.413 (28/06/2020)

WCS Keywords

Number of WCS axes: 2
CTYPE : 'RA---TAN'  'DEC--TAN'  
CRVAL : 74.7665419662444  -65.99868019807921  
CRPIX : 2111.0  2148.5  
CD1_1 CD1_2  : -1.1013595519102599e-05  8.461793445216233e-06  
CD2_1 CD2_2  : 8.461793445216233e-06  1.1013595519102599e-05  
NAXIS : 4222  4297
********************************************************************************
*
*  Estimated memory usage:  up to 1086 Mb.
*  Output image size:       4222 X 4297 pixels. 
*  Output image file:       ~ 207 Mb. 
*  Cores available:         4
*
********************************************************************************
==== Processing Step Initialization finished at 18:06:16.385 (28/06/2020)
==== Processing Step  Static Mask  started at  18:06:16.388 (28/06/2020)

==== Processing Step Static Ma


AstroDrizzle Version 3.1.6 is finished processing at 18:06:47.592 (28/06/2020).



   --------------------          --------------------
                   Step          Elapsed time
   --------------------          --------------------

         Initialization          0.9721 sec.
            Static Mask          0.9053 sec.
           Subtract Sky          2.0672 sec.
       Separate Drizzle          2.9759 sec.
          Create Median          4.8385 sec.
                   Blot          6.1960 sec.
                Driz_CR          3.1162 sec.
          Final Drizzle          11.0868 sec.
   ====================          ====================
                  Total          32.1580 sec.

Trailer file written to:  astrodrizzle.log
Setting up logfile :  astrodrizzle.log
AstroDrizzle Version 3.1.6 (2020-02-19 16:16:30 -0500) started at: 18:06:47.695 (28/06/2020)

==== Processing Step  Initialization  started at  18:06:47.69 (28/06/2020)

WCS Keywords

Number of WCS axes: 2
CTYPE : 'RA

==== Processing Step Driz_CR finished at 18:07:08.977 (28/06/2020)
==== Processing Step  Final Drizzle  started at  18:07:08.986 (28/06/2020)

WCS Keywords

Number of WCS axes: 2
CTYPE : 'RA---TAN'  'DEC--TAN'  
CRVAL : 75.58993055740775  -67.98948427840403  
CRPIX : 2109.0  2150.5  
CD1_1 CD1_2  : -4.6295638331194224e-06  -1.3094593284366408e-05  
CD2_1 CD2_2  : -1.3094593284366408e-05  4.6295638331194224e-06  
NAXIS : 4218  4301
-Generating simple FITS output: final_drc_sci.fits
Writing out image to disk: final_drc_sci.fits
Writing out image to disk: final_drc_wht.fits
Writing out image to disk: final_drc_ctx.fits
==== Processing Step Final Drizzle finished at 18:07:20.298 (28/06/2020)

AstroDrizzle Version 3.1.6 is finished processing at 18:07:20.300 (28/06/2020).



   --------------------          --------------------
                   Step          Elapsed time
   --------------------          --------------------

         Initialization          0.8064 sec.
            Static 

-Generating simple FITS output: /Users/jameskwon/Documents/Research/Data/deepcr/f435w/12103/a3/jbfla3l1q_sci2_blt.fits
Writing out image to disk: /Users/jameskwon/Documents/Research/Data/deepcr/f435w/12103/a3/jbfla3l1q_sci2_blt.fits
==== Processing Step Blot finished at 18:07:38.16 (28/06/2020)
==== Processing Step  Driz_CR  started at  18:07:38.165 (28/06/2020)

Creating output: /Users/jameskwon/Documents/Research/Data/deepcr/f435w/12103/a3/jbfla3kyq_sci1_crmask.fits
Creating output: /Users/jameskwon/Documents/Research/Data/deepcr/f435w/12103/a3/jbfla3l1q_sci1_crmask.fits
Creating output: /Users/jameskwon/Documents/Research/Data/deepcr/f435w/12103/a3/jbfla3kyq_sci2_crmask.fits
Creating output: /Users/jameskwon/Documents/Research/Data/deepcr/f435w/12103/a3/jbfla3l1q_sci2_crmask.fits
==== Processing Step Driz_CR finished at 18:07:41.648 (28/06/2020)
==== Processing Step  Final Drizzle  started at  18:07:41.656 (28/06/2020)

WCS Keywords

Number of WCS axes: 2
CTYPE : 'RA---TAN'  'DEC--T

-Generating simple FITS output: /Users/jameskwon/Documents/Research/Data/deepcr/f435w/10631/1/j9df01juq_single_sci.fits
Writing out image to disk: /Users/jameskwon/Documents/Research/Data/deepcr/f435w/10631/1/j9df01jpq_single_sci.fits
Writing out image to disk: /Users/jameskwon/Documents/Research/Data/deepcr/f435w/10631/1/j9df01jqq_single_sci.fits
Writing out image to disk: /Users/jameskwon/Documents/Research/Data/deepcr/f435w/10631/1/j9df01jsq_single_sci.fits
Writing out image to disk: /Users/jameskwon/Documents/Research/Data/deepcr/f435w/10631/1/j9df01juq_single_sci.fits
Writing out image to disk: /Users/jameskwon/Documents/Research/Data/deepcr/f435w/10631/1/j9df01jpq_single_wht.fits
Writing out image to disk: /Users/jameskwon/Documents/Research/Data/deepcr/f435w/10631/1/j9df01jqq_single_wht.fits
Writing out image to disk: /Users/jameskwon/Documents/Research/Data/deepcr/f435w/10631/1/j9df01jsq_single_wht.fits
Writing out image to disk: /Users/jameskwon/Documents/Research/Data/deepcr/

...



-----  Computing sky values requested image extensions (detector chips):  -----

   *   Image:   'j6lp03xlq_flc.fits['SCI',1,2]'  --  SKY = 37.11782567641314 (brightness units)
       Sky change (data units):
      - EXT = 'SCI',1   delta(MDRIZSKY) = 31.5502   NEW MDRIZSKY = 31.5502
      - EXT = 'SCI',2   delta(MDRIZSKY) = 31.5502   NEW MDRIZSKY = 31.5502
   *   Image:   'j6lp03xzq_flc.fits['SCI',1,2]'  --  SKY = 51.85055335362752 (brightness units)
       Sky change (data units):
      - EXT = 'SCI',1   delta(MDRIZSKY) = 1.55552   NEW MDRIZSKY = 1.55552
      - EXT = 'SCI',2   delta(MDRIZSKY) = 1.55552   NEW MDRIZSKY = 1.55552
   *   Image:   'j6lp03y6q_flc.fits['SCI',1,2]'  --  SKY = 57.533421235926 (brightness units)
       Sky change (data units):
      - EXT = 'SCI',1   delta(MDRIZSKY) = 48.9034   NEW MDRIZSKY = 48.9034
      - EXT = 'SCI',2   delta(MDRIZSKY) = 48.9034   NEW MDRIZSKY = 48.9034
   *   Image:   'j6lp03ysq_flc.fits['SCI',1,2]'  --  SKY = 33.65459666532628 (brightn

==== Processing Step  Final Drizzle  started at  18:18:32.140 (28/06/2020)

WCS Keywords

Number of WCS axes: 2
CTYPE : 'RA---TAN'  'DEC--TAN'  
CRVAL : 201.61972360030688  -47.533165014319316  
CRPIX : 2113.5  2268.5  
CD1_1 CD1_2  : -1.434572899389081e-06  1.3814602244155985e-05  
CD2_1 CD2_2  : 1.3814602244155985e-05  1.434572899389081e-06  
NAXIS : 4227  4537
-Generating simple FITS output: final_drc_sci.fits
Writing out image to disk: final_drc_sci.fits
Writing out image to disk: final_drc_wht.fits
Writing out image to disk: final_drc_ctx.fits
==== Processing Step Final Drizzle finished at 18:18:52.278 (28/06/2020)

AstroDrizzle Version 3.1.6 is finished processing at 18:18:52.280 (28/06/2020).



   --------------------          --------------------
                   Step          Elapsed time
   --------------------          --------------------

         Initialization          1.5762 sec.
            Static Mask          1.6268 sec.
           Subtract Sky          3.6600 sec

Writing out image to disk: /Users/jameskwon/Documents/Research/Data/deepcr/f435w/9442/5/j6lp05wvq_single_wht.fits
==== Processing Step Separate Drizzle finished at 18:19:03.626 (28/06/2020)
==== Processing Step  Create Median  started at  18:19:03.630 (28/06/2020)

reference sky value for image '/Users/jameskwon/Documents/Research/Data/deepcr/f435w/9442/5/j6lp05vuq_flc.fits' is 172.93067932128906
reference sky value for image '/Users/jameskwon/Documents/Research/Data/deepcr/f435w/9442/5/j6lp05w3q_flc.fits' is 8.01970100402832
reference sky value for image '/Users/jameskwon/Documents/Research/Data/deepcr/f435w/9442/5/j6lp05wcq_flc.fits' is 183.08689880371094
reference sky value for image '/Users/jameskwon/Documents/Research/Data/deepcr/f435w/9442/5/j6lp05wvq_flc.fits' is 178.36720275878906
Saving output median image to: 'final_med.fits'
==== Processing Step Create Median finished at 18:19:11.987 (28/06/2020)
==== Processing Step  Blot  started at  18:19:11.987 (28/06/2020)

    Blot: cr


   **  Input image: 'jc8mb1nds_flc.fits'
       EXT: 'SCI',1;	MASK: jc8mb1nds_skymatch_mask_sci1.fits[0]
       EXT: 'SCI',2;	MASK: jc8mb1nds_skymatch_mask_sci2.fits[0]

   **  Input image: 'jc8mb1njs_flc.fits'
       EXT: 'SCI',1;	MASK: jc8mb1njs_skymatch_mask_sci1.fits[0]
       EXT: 'SCI',2;	MASK: jc8mb1njs_skymatch_mask_sci2.fits[0]

   **  Input image: 'jc8mb1nqq_flc.fits'
       EXT: 'SCI',1;	MASK: jc8mb1nqq_skymatch_mask_sci1.fits[0]
       EXT: 'SCI',2;	MASK: jc8mb1nqq_skymatch_mask_sci2.fits[0]

   **  Input image: 'jc8mb1nxq_flc.fits'
       EXT: 'SCI',1;	MASK: jc8mb1nxq_skymatch_mask_sci1.fits[0]
       EXT: 'SCI',2;	MASK: jc8mb1nxq_skymatch_mask_sci2.fits[0]

-----  Sky statistics parameters:  -----
       statistics function: 'median'
       lower = None
       upper = None
       nclip = 5
       lsigma = 4.0
       usigma = 4.0
       binwidth = 0.1

-----  Data->Brightness conversion parameters for input files:  -----

   *   Image: jc8mb1nds_flc.fits
       EXT = 'SCI

-Generating simple FITS output: /Users/jameskwon/Documents/Research/Data/deepcr/f435w/13498/b1/jc8mb1nqq_sci1_blt.fits
Writing out image to disk: /Users/jameskwon/Documents/Research/Data/deepcr/f435w/13498/b1/jc8mb1nqq_sci1_blt.fits
    Blot: creating blotted image:  /Users/jameskwon/Documents/Research/Data/deepcr/f435w/13498/b1/jc8mb1nqq_flc.fits[sci,2]
Using default C-based coordinate transformation...
-Generating simple FITS output: /Users/jameskwon/Documents/Research/Data/deepcr/f435w/13498/b1/jc8mb1nqq_sci2_blt.fits
Writing out image to disk: /Users/jameskwon/Documents/Research/Data/deepcr/f435w/13498/b1/jc8mb1nqq_sci2_blt.fits
    Blot: creating blotted image:  /Users/jameskwon/Documents/Research/Data/deepcr/f435w/13498/b1/jc8mb1nxq_flc.fits[sci,1]
Using default C-based coordinate transformation...
-Generating simple FITS output: /Users/jameskwon/Documents/Research/Data/deepcr/f435w/13498/b1/jc8mb1nxq_sci1_blt.fits
Writing out image to disk: /Users/jameskwon/Documents/Research/Da

In [199]:
for prop_id in os.listdir(f814_dir):
    sub_dir = os.path.join(f814_dir,prop_id)
    if os.path.isdir(sub_dir):
        if prop_id != 'mastDownload':
            for visit_num in os.listdir(sub_dir):
                if visit_num != '.DS_Store':
                    subsub_dir = os.path.join(sub_dir,visit_num)
                    drizzle_path = subsub_dir + '/*flc.fits'
                    astrodrizzle.AstroDrizzle(drizzle_path)
                    remove_extra(subsub_dir)

Setting up logfile :  astrodrizzle.log
AstroDrizzle Version 3.1.6 (2020-02-19 16:16:30 -0500) started at: 18:22:13.605 (28/06/2020)

==== Processing Step  Initialization  started at  18:22:13.609 (28/06/2020)

WCS Keywords

Number of WCS axes: 2
CTYPE : 'RA---TAN'  'DEC--TAN'  
CRVAL : 228.5518797422735  -46.81058614818798  
CRPIX : 2107.0  2118.0  
CD1_1 CD1_2  : 7.92411794770195e-06  1.1406559048144583e-05  
CD2_1 CD2_2  : 1.1406559048144583e-05  -7.92411794770195e-06  
NAXIS : 4214  4236
********************************************************************************
*
*  Estimated memory usage:  up to 1073 Mb.
*  Output image size:       4214 X 4236 pixels. 
*  Output image file:       ~ 204 Mb. 
*  Cores available:         4
*
********************************************************************************
==== Processing Step Initialization finished at 18:22:14.55 (28/06/2020)
==== Processing Step  Static Mask  started at  18:22:14.555 (28/06/2020)

==== Processing Step Static Ma


AstroDrizzle Version 3.1.6 is finished processing at 18:22:46.138 (28/06/2020).



   --------------------          --------------------
                   Step          Elapsed time
   --------------------          --------------------

         Initialization          0.9433 sec.
            Static Mask          1.0273 sec.
           Subtract Sky          1.9200 sec.
       Separate Drizzle          2.7953 sec.
          Create Median          4.4642 sec.
                   Blot          5.7790 sec.
                Driz_CR          3.4442 sec.
          Final Drizzle          12.1320 sec.
   ====================          ====================
                  Total          32.5053 sec.

Trailer file written to:  astrodrizzle.log
Setting up logfile :  astrodrizzle.log
AstroDrizzle Version 3.1.6 (2020-02-19 16:16:30 -0500) started at: 18:22:46.249 (28/06/2020)

==== Processing Step  Initialization  started at  18:22:46.251 (28/06/2020)

WCS Keywords

Number of WCS axes: 2
CTYPE : 'R

Creating output: /Users/jameskwon/Documents/Research/Data/deepcr/f814w/10235/15/j8xz15bjq_sci2_crmask.fits
==== Processing Step Driz_CR finished at 18:23:07.26 (28/06/2020)
==== Processing Step  Final Drizzle  started at  18:23:07.276 (28/06/2020)

WCS Keywords

Number of WCS axes: 2
CTYPE : 'RA---TAN'  'DEC--TAN'  
CRVAL : 225.28142570008342  -48.29020589348107  
CRPIX : 2107.0  2118.5  
CD1_1 CD1_2  : 8.667671406362865e-06  1.0852313438120961e-05  
CD2_1 CD2_2  : 1.0852313438120961e-05  -8.667671406362865e-06  
NAXIS : 4214  4237
-Generating simple FITS output: final_drc_sci.fits
Writing out image to disk: final_drc_sci.fits
Writing out image to disk: final_drc_wht.fits
Writing out image to disk: final_drc_ctx.fits
==== Processing Step Final Drizzle finished at 18:23:19.390 (28/06/2020)

AstroDrizzle Version 3.1.6 is finished processing at 18:23:19.39 (28/06/2020).



   --------------------          --------------------
                   Step          Elapsed time
   --------------

Writing out image to disk: /Users/jameskwon/Documents/Research/Data/deepcr/f814w/12058/6/jbf306nyq_single_wht.fits
Writing out image to disk: /Users/jameskwon/Documents/Research/Data/deepcr/f814w/12058/6/jbf306nuq_single_wht.fits
Writing out image to disk: /Users/jameskwon/Documents/Research/Data/deepcr/f814w/12058/6/jbf306noq_single_wht.fits
==== Processing Step Separate Drizzle finished at 18:23:31.208 (28/06/2020)
==== Processing Step  Create Median  started at  18:23:31.212 (28/06/2020)

reference sky value for image '/Users/jameskwon/Documents/Research/Data/deepcr/f814w/12058/6/jbf306noq_flc.fits' is 29.582294464111328
reference sky value for image '/Users/jameskwon/Documents/Research/Data/deepcr/f814w/12058/6/jbf306nrq_flc.fits' is 686.0902099609375
reference sky value for image '/Users/jameskwon/Documents/Research/Data/deepcr/f814w/12058/6/jbf306nuq_flc.fits' is 1387.0960693359375
reference sky value for image '/Users/jameskwon/Documents/Research/Data/deepcr/f814w/12058/6/jbf306

'MDRIZSKY' header keyword will represent sky value *computed* from data.

-----  User specified keywords:  -----
       Sky Value Keyword:  'MDRIZSKY'
       Data Units Keyword: 'BUNIT'


-----  Input file list:  -----

   **  Input image: 'jbf310olq_flc.fits'
       EXT: 'SCI',1;	MASK: jbf310olq_skymatch_mask_sci1.fits[0]
       EXT: 'SCI',2;	MASK: jbf310olq_skymatch_mask_sci2.fits[0]

   **  Input image: 'jbf310ooq_flc.fits'
       EXT: 'SCI',1;	MASK: jbf310ooq_skymatch_mask_sci1.fits[0]
       EXT: 'SCI',2;	MASK: jbf310ooq_skymatch_mask_sci2.fits[0]

   **  Input image: 'jbf310orq_flc.fits'
       EXT: 'SCI',1;	MASK: jbf310orq_skymatch_mask_sci1.fits[0]
       EXT: 'SCI',2;	MASK: jbf310orq_skymatch_mask_sci2.fits[0]

   **  Input image: 'jbf310ovq_flc.fits'
       EXT: 'SCI',1;	MASK: jbf310ovq_skymatch_mask_sci1.fits[0]
       EXT: 'SCI',2;	MASK: jbf310ovq_skymatch_mask_sci2.fits[0]

-----  Sky statistics parameters:  -----
       statistics function: 'median'
       lower = None
  

...

==== Processing Step  Create Median  started at  18:30:56.01 (28/06/2020)

reference sky value for image '/Users/jameskwon/Documents/Research/Data/deepcr/f814w/13024/16/jbz016mgq_flc.fits' is 31.064353942871094
reference sky value for image '/Users/jameskwon/Documents/Research/Data/deepcr/f814w/13024/16/jbz016mhq_flc.fits' is 41.340145111083984
Saving output median image to: 'final_med.fits'
==== Processing Step Create Median finished at 18:31:00.719 (28/06/2020)
==== Processing Step  Blot  started at  18:31:00.72 (28/06/2020)

    Blot: creating blotted image:  /Users/jameskwon/Documents/Research/Data/deepcr/f814w/13024/16/jbz016mgq_flc.fits[sci,1]
Using default C-based coordinate transformation...
-Generating simple FITS output: /Users/jameskwon/Documents/Research/Data/deepcr/f814w/13024/16/jbz016mgq_sci1_blt.fits
Writing out image to disk: /Users/jameskwon/Documents/Research/Data/deepcr/f814w/13024/16/jbz016mgq_sci1_blt.fits
    Blot: creating blotted image:  /Users/jameskwon/Docume

NAXIS : 4223  4297
-Generating simple FITS output: /Users/jameskwon/Documents/Research/Data/deepcr/f814w/13024/10/jbz010dpq_single_sci.fits
-Generating simple FITS output: /Users/jameskwon/Documents/Research/Data/deepcr/f814w/13024/10/jbz010doq_single_sci.fits
Writing out image to disk: /Users/jameskwon/Documents/Research/Data/deepcr/f814w/13024/10/jbz010dpq_single_sci.fits
Writing out image to disk: /Users/jameskwon/Documents/Research/Data/deepcr/f814w/13024/10/jbz010doq_single_sci.fits
Writing out image to disk: /Users/jameskwon/Documents/Research/Data/deepcr/f814w/13024/10/jbz010dpq_single_wht.fits
Writing out image to disk: /Users/jameskwon/Documents/Research/Data/deepcr/f814w/13024/10/jbz010doq_single_wht.fits
==== Processing Step Separate Drizzle finished at 18:31:29.369 (28/06/2020)
==== Processing Step  Create Median  started at  18:31:29.372 (28/06/2020)

reference sky value for image '/Users/jameskwon/Documents/Research/Data/deepcr/f814w/13024/10/jbz010doq_flc.fits' is 31.590


   *   Image: j8xi01w3q_flc.fits
       EXT = 'SCI',1
             Data units type: COUNTS
             EXPTIME: 507.0 [s]
             Conversion factor (data->brightness):  0.7889546351084812
       EXT = 'SCI',2
             Data units type: COUNTS
             EXPTIME: 507.0 [s]
             Conversion factor (data->brightness):  0.7889546351084812

   *   Image: j8xi01w7q_flc.fits
       EXT = 'SCI',1
             Data units type: COUNTS
             EXPTIME: 507.0 [s]
             Conversion factor (data->brightness):  0.7889546351084812
       EXT = 'SCI',2
             Data units type: COUNTS
             EXPTIME: 507.0 [s]
             Conversion factor (data->brightness):  0.7889546351084812


-----  Computing sky values requested image extensions (detector chips):  -----

   *   Image:   'j8xi01vxq_flc.fits['SCI',1,2]'  --  SKY = 40.46372332751633 (brightness units)
       Sky change (data units):
      - EXT = 'SCI',1   delta(MDRIZSKY) = 51.2878   NEW MDRIZSKY = 51.2878
  

Creating output: /Users/jameskwon/Documents/Research/Data/deepcr/f814w/10092/1/j8xi01w0q_sci1_crmask.fits
Creating output: /Users/jameskwon/Documents/Research/Data/deepcr/f814w/10092/1/j8xi01w7q_sci1_crmask.fits
Creating output: /Users/jameskwon/Documents/Research/Data/deepcr/f814w/10092/1/j8xi01vxq_sci2_crmask.fits
Creating output: /Users/jameskwon/Documents/Research/Data/deepcr/f814w/10092/1/j8xi01w3q_sci2_crmask.fits
Creating output: /Users/jameskwon/Documents/Research/Data/deepcr/f814w/10092/1/j8xi01w0q_sci2_crmask.fits
Creating output: /Users/jameskwon/Documents/Research/Data/deepcr/f814w/10092/1/j8xi01w7q_sci2_crmask.fits
==== Processing Step Driz_CR finished at 18:32:31.488 (28/06/2020)
==== Processing Step  Final Drizzle  started at  18:32:31.497 (28/06/2020)

WCS Keywords

Number of WCS axes: 2
CTYPE : 'RA---TAN'  'DEC--TAN'  
CRVAL : 150.67386384385944  2.5938625401367217  
CRPIX : 2112.0  2211.0  
CD1_1 CD1_2  : 2.411613805671565e-06  1.3677914805268951e-05  
CD2_1 CD2_2  : 

reference sky value for image '/Users/jameskwon/Documents/Research/Data/deepcr/f814w/13737/2/jcoy02cbq_flc.fits' is 170.5322570800781
reference sky value for image '/Users/jameskwon/Documents/Research/Data/deepcr/f814w/13737/2/jcoy02cfq_flc.fits' is 140.78280639648438
Saving output median image to: 'final_med.fits'
==== Processing Step Create Median finished at 18:33:09.065 (28/06/2020)
==== Processing Step  Blot  started at  18:33:09.066 (28/06/2020)

    Blot: creating blotted image:  /Users/jameskwon/Documents/Research/Data/deepcr/f814w/13737/2/jcoy02c8q_flc.fits[sci,1]
Using default C-based coordinate transformation...
-Generating simple FITS output: /Users/jameskwon/Documents/Research/Data/deepcr/f814w/13737/2/jcoy02c8q_sci1_blt.fits
Writing out image to disk: /Users/jameskwon/Documents/Research/Data/deepcr/f814w/13737/2/jcoy02c8q_sci1_blt.fits
    Blot: creating blotted image:  /Users/jameskwon/Documents/Research/Data/deepcr/f814w/13737/2/jcoy02c8q_flc.fits[sci,2]
Using default C

## Bad Pixel Masks and Saturation Masks - Getting Bad pixel masks and saturation masks

In [14]:
tr_prop_ID = [10420,12213,10775,10260,10584]

In [54]:
img_pair = {}
for i,ID in enumerate(tr_prop_ID):
    print(f'{i+1}/{len(tr_prop_ID)}', end = '\r')
    parent_path = f'{tr_path}{ID}'
    n_vis = !ls $parent_path
    for n in n_vis:
        fits_path = f'{parent_path}/{n}'
        fits_files = !ls $fits_path
        for file in fits_files:
            if '_crmask.fits' not in file:
                with fits.open(f'{fits_path}/{file}') as hdul:
                    dq = hdul[3].data
                    img = hdul[1].data
                    sat = img > 70000
                    dq = (dq > 0) & (dq != 2048) & (dq != 256)
                    
                    img2 = hdul[4].data
                    sat2 = img2> 70000
                    dq2 = hdul[6].data
                    dq2 = (dq2 > 0) & (dq2 != 2048) & (dq2 != 256)

                f_crmask = file.replace('_flc.fits','_sci1_crmask.fits')
                f_crmask2 = file.replace('_flc.fits','_sci2_crmask.fits')
                crmask = fits.getdata(f'{fits_path}/{f_crmask}')
                crmask2 = fits.getdata(f'{fits_path}/{f_crmask2}')
                
                ign1 = dq | sat
                ign2 = dq2 | sat2
                
                ign1 = np.array(ign1, dtype = int)
                ign2 = np.array(ign2, dtype = int)
                
                img_pair[f'{file}_1'] = (img,crmask, ign1)
                img_pair[f'{file}_2'] = (img2,crmask2, ign2)
                del dq, img, sat,crmask, dq2, img2, sat2, crmask2, ign1, ign2
np.save('img_pair.npy',img_pair)

## Train - the image, masks, ignores are then segmented to 256x256 below

In [55]:
img_pair = np.load('img_pair.npy', allow_pickle = True)[()]

In [56]:
length = 256

subimg_pair = {}
for i,pair in enumerate(img_pair.items()):
    print(f'{i+1}/{len(img_pair)}', end = '\r')
    f_name, val = pair
    img, mask, ignore = val
    y, x = img.shape
    y /= length
    x /= length
    idx = 0
    for yy in range(int(y)):
        for xx in range(int(x)):
            sub_img = img[length*(yy):length*(yy+1),length*(xx):length*(xx+1)]
            sub_mask = mask[length*(yy):length*(yy+1),length*(xx):length*(xx+1)]
            sub_ignore = ignore[length*(yy):length*(yy+1),length*(xx):length*(xx+1)]
            subimg_pair[f'{f_name}_{idx}'] = sub_img,sub_mask,sub_ignore
            idx += 1
np.save('subimg_pair.npy',subimg_pair)

In [3]:
subimg_pair = np.load('subimg_pair.npy',allow_pickle = True)[()]

In [57]:
ts_prop_ID = [13671,10775,10190]

In [58]:
img_pair = {}
for i,ID in enumerate(ts_prop_ID):
    print(f'{i+1}/{len(ts_prop_ID)}', end = '\r')
    parent_path = f'{ts_path}{ID}'
    n_vis = !ls $parent_path
    for n in n_vis:
        fits_path = f'{parent_path}/{n}'
        fits_files = !ls $fits_path
        for file in fits_files:
            if '_crmask.fits' not in file:
                with fits.open(f'{fits_path}/{file}') as hdul:
                    dq = hdul[3].data
                    img = hdul[1].data
                    sat = img > 70000
                    dq = (dq > 0) & (dq != 2048) & (dq != 256)
                    
                    img2 = hdul[4].data
                    sat2 = img2> 70000
                    dq2 = hdul[6].data
                    dq2 = (dq2 > 0) & (dq2 != 2048) & (dq2 != 256)

                f_crmask = file.replace('_flc.fits','_sci1_crmask.fits')
                f_crmask2 = file.replace('_flc.fits','_sci2_crmask.fits')
                crmask = fits.getdata(f'{fits_path}/{f_crmask}')
                crmask2 = fits.getdata(f'{fits_path}/{f_crmask2}')
                
                ign1 = dq | sat
                ign2 = dq2 | sat2
                
                ign1 = np.array(ign1, dtype = int)
                ign2 = np.array(ign2, dtype = int)
                
                img_pair[f'{file}_1'] = (img,crmask, ign1)
                img_pair[f'{file}_2'] = (img2,crmask2, ign2)
                del dq, img, sat,crmask, dq2, img2, sat2, crmask2, ign1, ign2
np.save('ts_img_pair.npy',img_pair)

In [59]:
length = 256

subimg_pair = {}
for i,pair in enumerate(img_pair.items()):
    print(f'{i+1}/{len(img_pair)}', end = '\r')
    f_name, val = pair
    img, mask, ignore = val
    y, x = img.shape
    y /= length
    x /= length
    idx = 0
    for yy in range(int(y)):
        for xx in range(int(x)):
            sub_img = img[length*(yy):length*(yy+1),length*(xx):length*(xx+1)]
            sub_mask = mask[length*(yy):length*(yy+1),length*(xx):length*(xx+1)]
            sub_ignore = ignore[length*(yy):length*(yy+1),length*(xx):length*(xx+1)]
            subimg_pair[f'{f_name}_{idx}'] = sub_img,sub_mask,sub_ignore
            idx += 1
np.save('ts_subimg_pair.npy',subimg_pair)